In [ ]:
from functools import partial
import numpy as np
import tensorflow as tf
import math

In [ ]:
#Import training data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('mnist_data', one_hot=True, reshape=False)

In [ ]:
#Create variables
x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='InputData')
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')
# step for variable learning rate
step = tf.placeholder(tf.int32)
drop_rate = tf.placeholder(tf.float32)
batch_momentum = 0.9
training = tf.placeholder_with_default(False, shape=(), name="training") # for BN

In [ ]:
# Create partial bn function
batch_normalization_layer = partial(tf.layers.batch_normalization, training = training, momentum = 0.9)

In [ ]:
# Convolutional Layer #1
conv1 = tf.layers.conv2d(inputs = x, filters = 32, kernel_size = [5, 5], padding = "same", activation = None)
conv1_bn = batch_normalization_layer(conv1)
conv1_bn_relu = tf.nn.relu(conv1_bn)

# Pooling Layer #1
pool1 = tf.layers.max_pooling2d(inputs = conv1_bn_relu, pool_size = [2, 2], strides = 2)

In [ ]:
# Convolutional Layer #2
conv2 = tf.layers.conv2d(inputs = pool1, filters = 64, kernel_size = [5, 5], padding = "same", activation = None)
conv2_bn = batch_normalization_layer(conv2)
conv2_bn_relu = tf.nn.relu(conv2_bn)

# Pooling Layer #2
pool2 = tf.layers.max_pooling2d(inputs = conv2_bn_relu, pool_size = [2, 2], strides = 2)

In [ ]:
# Dense Layer
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs = pool2_flat, units = 1024, activation = tf.nn.relu)
dropout = tf.layers.dropout(inputs = dense, rate = drop_rate)

In [ ]:
# Logits Layer
Y = tf.layers.dense(inputs = dropout, units = 10)
Ys = tf.nn.softmax(Y)

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=y)
cross_entropy = tf.reduce_mean(cross_entropy) * 100.0

lr = 0.0001 +  tf.train.exponential_decay(0.003, step, 2000, 1/math.e)

optimizer = tf.train.AdamOptimizer(lr)
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
tf.control_dependencies(extra_update_ops)
train_step = optimizer.minimize(cross_entropy)

# Accuracy
acc = tf.equal(tf.argmax(Y, 1), tf.argmax(y, 1))
acc = tf.reduce_mean(tf.cast(acc, tf.float32))

In [ ]:
epochs = 1000
batch_size = 200
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(epochs):
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, drop_rate: 0.75, step: iteration * epoch + 1, training:True})
            if iteration % 100 == 0:
                print("Epoka: " + str(epoch) + "  Iteracja: " + str(iteration))
                print(acc.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels, drop_rate:1.0}))


#sess = tf.InteractiveSession()
#tf.global_variables_initializer().run()

#for i in range(10000):
#    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#    train_data = {x: batch_xs, y: batch_ys, drop_rate: 0.5, step: i}
#    sess.run(train_step, feed_dict=train_data)

#print(acc.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels, drop_rate:1.0}))